In [1]:
import paddle as paddle
import paddle.fluid as fluid
import numpy as np
import matplotlib.pyplot as plt
import sys
import cv2
import os
import random
import math
import PIL.Image as Image
from paddle.fluid.initializer import MSRA
from paddle.fluid.param_attr import ParamAttr
from visualdl import LogWriter

In [2]:
def produce_path_label(path):
    train_list = np.load(path)
    return train_list

def produce_test(path):
    train_list = []
    train_data = open(path).readlines()
    for i in train_data:
        items = i.split(',')
        class_index = int(items[0])
        img = np.array(items[1:]).astype('uint8')
        train_list.append([class_index,img])
    return train_list

train_list = produce_path_label('data/Martix_train_list.npy')
test_list = produce_path_label('data/Martix_test_list.npy')
class_number = len(train_list)
print(class_number)

21


In [3]:
data_shape = [64,64]
class AlexNet():
    def __init__(self):
        pass

    def net(self, input, class_dim=1000):
        stdv = 1.0 / math.sqrt(input.shape[1] * 5 * 5)
        conv1 = fluid.layers.conv2d(
            input=input,
            num_filters=64,
            filter_size=5,
            stride=1,
            padding=2,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))
        pool1 = fluid.layers.pool2d(
            input=conv1,
            pool_size=3,
            pool_stride=2,
            pool_padding=0,
            pool_type='max')

        stdv = 1.0 / math.sqrt(pool1.shape[1] * 5 * 5)
        conv2 = fluid.layers.conv2d(
            input=pool1,
            num_filters=192,
            filter_size=5,
            stride=1,
            padding=2,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))
        pool2 = fluid.layers.pool2d(
            input=conv2,
            pool_size=3,
            pool_stride=2,
            pool_padding=0,
            pool_type='max')

        stdv = 1.0 / math.sqrt(pool2.shape[1] * 3 * 3)
        conv3 = fluid.layers.conv2d(
            input=pool2,
            num_filters=384,
            filter_size=3,
            stride=1,
            padding=1,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))

        stdv = 1.0 / math.sqrt(conv3.shape[1] * 3 * 3)
        conv4 = fluid.layers.conv2d(
            input=conv3,
            num_filters=256,
            filter_size=3,
            stride=1,
            padding=1,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))

        stdv = 1.0 / math.sqrt(conv4.shape[1] * 3 * 3)
        conv5 = fluid.layers.conv2d(
            input=conv4,
            num_filters=256,
            filter_size=3,
            stride=1,
            padding=1,
            groups=1,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))
        pool5 = fluid.layers.pool2d(
            input=conv5,
            pool_size=3,
            pool_stride=2,
            pool_padding=0,
            pool_type='max')
        print(pool5)
        drop6 = fluid.layers.dropout(x=pool5, dropout_prob=0.5)

        stdv = 1.0 / math.sqrt(drop6.shape[1] * drop6.shape[2] *
                               drop6.shape[3] * 1.0)
        fc6 = fluid.layers.fc(
            input=drop6,
            size=256,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))

        drop7 = fluid.layers.dropout(x=fc6, dropout_prob=0.5)

        stdv = 1.0 / math.sqrt(drop7.shape[1] * 1.0)
        fc7 = fluid.layers.fc(
            input=drop7,
            size=128,
            act='relu',
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)))

        stdv = 1.0 / math.sqrt(fc7.shape[1] * 1.0)
        out = fluid.layers.fc(
            input=fc7,
            size=class_dim,
            bias_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),
            param_attr=fluid.param_attr.ParamAttr(
                initializer=fluid.initializer.Uniform(-stdv, stdv)),act='softmax')
        return out

In [4]:

class ResNet():
    def __init__(self, layers=50):
        self.layers = layers

    def net(self, input, class_dim=1000):
        layers = self.layers
        if layers == 18:
            depth = [2, 2, 2, 2]
        if layers == 50:
            depth = [3, 4, 6, 3]
        elif layers == 101:
            depth = [3, 4, 23, 3]
        elif layers == 152:
            depth = [3, 8, 36, 3]
        num_filters = [64, 128, 256, 512]
        conv = self.conv_bn_layer(
            input=input, num_filters=64, filter_size=7, stride=2, act='relu')
        conv = fluid.layers.pool2d(
            input=conv,
            pool_size=3,
            pool_stride=2,
            pool_padding=1,
            pool_type='max')
        for block in range(len(depth)):
            for i in range(depth[block]):
                conv = self.bottleneck_block(
                    input=conv,
                    num_filters=num_filters[block],
                    stride=2 if i == 0 and block != 0 else 1)
        pool = fluid.layers.pool2d(
            input=conv, pool_size=7, pool_type='avg', global_pooling=True)
        stdv = 1.0 / math.sqrt(pool.shape[1] * 1.0)
        out = fluid.layers.fc(input=pool,
                              act = 'softmax',
                              size=class_dim,
                              param_attr=fluid.param_attr.ParamAttr(
                                  initializer=fluid.initializer.Uniform(-stdv,
                                                                        stdv)))
        return out
    def conv_bn_layer(self,
                      input,
                      num_filters,
                      filter_size,
                      stride=1,
                      groups=1,
                      act=None):
        conv = fluid.layers.conv2d(
            input=input,
            num_filters=num_filters,
            filter_size=filter_size,
            stride=stride,
            padding=(filter_size - 1) // 2,
            groups=groups,
            act=None,
            bias_attr=False)
        return fluid.layers.batch_norm(input=conv, act=act)

    def shortcut(self, input, ch_out, stride):
        ch_in = input.shape[1]
        if ch_in != ch_out or stride != 1:
            return self.conv_bn_layer(input, ch_out, 1, stride)
        else:
            return input

    def bottleneck_block(self, input, num_filters, stride):
        conv0 = self.conv_bn_layer(
            input=input, num_filters=num_filters, filter_size=1, act='relu')
        conv1 = self.conv_bn_layer(
            input=conv0,
            num_filters=num_filters,
            filter_size=3,
            stride=stride,
            act='relu')
        conv2 = self.conv_bn_layer(
            input=conv1, num_filters=num_filters * 4, filter_size=1, act=None)
        short = self.shortcut(input, num_filters * 4, stride)
        return fluid.layers.elementwise_add(x=short, y=conv2, act='relu')


def ResNet50():
    model = ResNet(layers=18)
    return model

In [ ]:
yuzhi = 1
def addGaussianNoise(image): 
    G_Noiseimg = np.array(image)
    G_NoiseNum=random.randint(1,5)
    for i in range(G_NoiseNum): 
        temp_x = np.random.randint(0,image.shape[0])
        temp_y = np.random.randint(0,image.shape[0])
        temp_x_size = np.random.randint(1,5)
        temp_y_size = np.random.randint(1,5)
        temp_x_end = min(image.shape[0],temp_x+temp_x_size)
        temp_y_end = min(image.shape[0],temp_y+temp_y_size)
        for x in range(temp_x,temp_x_end):
            for y in range(temp_y,temp_y_end):
                G_Noiseimg[x][y] = 255
    return G_Noiseimg
def for_iterater_reader(t_list):
    def reader():
        for i in range(0,4000):
            for label in range(0,class_number):
              #  try:
                    #img = cv2.resize(img,(data_shape[0],data_shape[1]))
                    #img = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                    #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                    tmp_ran = train_list[label]
                    ran_int = random.randint(0,len(tmp_ran)-1)#取随机的一个数
                    img = np.array(tmp_ran[ran_int]).reshape(64,64)
                    r_sofang = random.randint(6,12)/10
                    shape = int(r_sofang*data_shape[0])
                    if label == 17 or label == 16:
                        r_sofang = random.randint(6,8)/10
                        shape = int(r_sofang*data_shape[0])
                        e_sofang = random.randint(4,10)/10
                        img = Image.fromarray(img).resize((int(shape*e_sofang),shape))
                    else:
                        img = Image.fromarray(img).resize((shape,shape))
                    img = np.array(img)
                    _, img = cv2.threshold(img, 10, 255,cv2.THRESH_BINARY)
                    h,w = np.array(img).shape
                    r_x = random.randint(-5,5)
                    r_y = random.randint(-4,4)
                    r_rota = random.randint(-5,5)
                    r_yh = random.randint(1,3)
                    img = Image.fromarray(img)
                    img = img.rotate(r_rota)
                    tmpimg = Image.new('L',(data_shape[0],data_shape[0]))
                    tmpimg.paste(img,(r_x,r_y))
                    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (r_yh, r_yh))
                    tmpimg = cv2.dilate(np.array(tmpimg), element1, iterations = 1)
                    tmpimg = addGaussianNoise(tmpimg)
                    #img = cv2.blur(img,(20,20))#羽化
                    #ret, binary = cv2.threshold(img, 50, 255,cv2.THRESH_BINARY)
                    #element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (10,10))#拓展
                    #img = cv2.erode(binary, element1, iterations = 1)
                    img = np.array(tmpimg)
                    ######################################################
                    img = img.reshape(1,data_shape[0],data_shape[1]).astype('float32')
                    img = (img)/255 
                    yield label,img
            #    except Exception:
             #       print(label)
    return reader

def for_test_reader(t_list):
    def reader():
        for class_index in t_list:
            image = np.array(class_index[1]).reshape(64,64)
            image = Image.fromarray(image).resize((data_shape[0],data_shape[0]))
            image = np.array(image).reshape(1,data_shape[0],data_shape[0]).astype('float32')
            image = (image) / 255.0
            yield class_index[0],image
    return reader

In [ ]:
#1.设置地方
place = fluid.CUDAPlace(0)
#2.设置数据和设置标签
label = fluid.layers.data(name = 'label',shape=[1],dtype='int64')
image = fluid.layers.data(name = 'image',shape=[1,data_shape[0],data_shape[1]],dtype='float32')
#3.设置网络和Feeder
feeder = fluid.DataFeeder(place = place , feed_list = [label,image])
net = ResNet50().net(image,class_number)
#4.设置损失函数和正确率
cost = fluid.layers.cross_entropy(input = net , label = label)
avg_cost = fluid.layers.mean(cost)
acc = fluid.layers.accuracy(input = net , label =label)
#.定义测试程序
test_program = fluid.default_main_program().clone(for_test=True)
#5.设置优化
LR = 0.00001
piecewise_decay = fluid.layers.piecewise_decay([10000,20000,30000], [LR, LR * 0.01,LR * 0.001,LR * 0.0001])
optimizer = fluid.optimizer.Adam(learning_rate=piecewise_decay)
optimizer.minimize(avg_cost)
#6.定义Executor
exe = fluid.Executor(place = place)
exe.run(fluid.default_startup_program())

[]

In [ ]:
import shutil
import os
def save_inference(pass_id):
     # 保存预测模型
    save_path = 'models/infer_model/'+str(pass_id)+"/"
    # 删除旧的模型文件
    shutil.rmtree(save_path, ignore_errors=True)
    # 创建保持模型文件目录
    os.makedirs(save_path)
    # 保存预测模型
    fluid.io.save_inference_model(save_path, feeded_var_names=[image.name], target_vars=[net], executor=exe)

def save_model(pass_id):
    # 保存参数模型
    save_path = 'models/params_model/'+str(pass_id)+"/"
    # 删除旧的模型文件
    shutil.rmtree(save_path, ignore_errors=True)
    # 创建保持模型文件目录
    os.makedirs(save_path)
    # 保存参数模型
    fluid.io.save_params(executor=exe, dirname=save_path)

def load_model(pass_id):
    # 加载之前训练过的参数模型
    save_path = 'models/params_model/'+str(pass_id)
    if os.path.exists(save_path):
        print('使用参数模型作为预训练模型')
        fluid.io.load_params(executor=exe, dirname=save_path)

In [ ]:
#8.数据分批
train_Reader = paddle.batch(reader=paddle.reader.shuffle(for_iterater_reader(train_list),buf_size = 128*128),batch_size=64)
test_Reader = paddle.batch(reader=paddle.reader.shuffle(for_iterater_reader(test_list),buf_size = 128*128),batch_size=64)

In [ ]:
print("加载完毕")

加载完毕


In [ ]:
#9.训练
step = 0
best_model_precent = 0
error_cout = 20
save_model_name = 35

In [ ]:
load_model(save_model_name)

使用参数模型作为预训练模型


In [ ]:
 def train():
    for pass_id in range(20000):
        sum_cost=0
        sum_acc=0
        global best_model_precent
        for batch_id , data in enumerate(train_Reader()):
            train_cost, train_acc = exe.run(program = fluid.default_main_program(),
                                           feed = feeder.feed(data),
                                            fetch_list = [avg_cost,acc]
                                           )
            sum_cost = sum_cost + train_cost[0]
            sum_acc = sum_acc + train_acc[0]
            if batch_id % 50 == 0 and batch_id != 0:
                print('Pass：%d, Batch：%d, Cost：%f, Accuracy：%f' % (pass_id , batch_id , sum_cost/51, sum_acc/51))
                sum_cost = 0
                sum_acc = 0
        test_sum_cost=0
        test_sum_acc=0
        test_cout = 0
        for batch_id , data in enumerate(test_Reader()):
            train_cost, train_acc = exe.run(program = test_program,
                                           feed = feeder.feed(data),
                                            fetch_list = [avg_cost,acc]
                                           )
            test_sum_cost = test_sum_cost + train_cost[0]
            test_sum_acc = test_sum_acc + train_acc[0]
            test_cout+=1
        print("test - Pass:",pass_id,"Cost:",test_sum_cost/test_cout,"Acc:",test_sum_acc/test_cout)
        total_model_precent = (test_sum_acc/test_cout)
        print("new BestPrecent:",best_model_precent,"thie Precent:",total_model_precent)
        if total_model_precent > best_model_precent:
            best_model_precent = total_model_precent
            save_model(save_model_name)
            print("Save...model...Best_precent:",best_model_precent)
        print()
train()

Pass：0, Batch：50, Cost：0.018847, Accuracy：0.993566
Pass：0, Batch：100, Cost：0.010223, Accuracy：0.977328
Pass：0, Batch：150, Cost：0.018510, Accuracy：0.973346
Pass：0, Batch：200, Cost：0.014700, Accuracy：0.974877
Pass：0, Batch：250, Cost：0.019915, Accuracy：0.974877
Pass：0, Batch：300, Cost：0.017654, Accuracy：0.974571
Pass：0, Batch：350, Cost：0.021628, Accuracy：0.973652
Pass：0, Batch：400, Cost：0.017157, Accuracy：0.975184
Pass：0, Batch：450, Cost：0.015010, Accuracy：0.975797
Pass：0, Batch：500, Cost：0.015253, Accuracy：0.974877
Pass：0, Batch：550, Cost：0.018355, Accuracy：0.973958
Pass：0, Batch：600, Cost：0.018069, Accuracy：0.974265
Pass：0, Batch：650, Cost：0.017068, Accuracy：0.973958
Pass：0, Batch：700, Cost：0.011381, Accuracy：0.976716
Pass：0, Batch：750, Cost：0.018894, Accuracy：0.974877
Pass：0, Batch：800, Cost：0.020574, Accuracy：0.973346
Pass：0, Batch：850, Cost：0.018970, Accuracy：0.974265
Pass：0, Batch：900, Cost：0.020161, Accuracy：0.974265
Pass：0, Batch：950, Cost：0.017423, Accuracy：0.974877
Pass：0, Batch

Pass：5, Batch：650, Cost：0.012387, Accuracy：0.976103
Pass：5, Batch：700, Cost：0.012649, Accuracy：0.977022
Pass：5, Batch：750, Cost：0.011776, Accuracy：0.976103
Pass：5, Batch：800, Cost：0.014841, Accuracy：0.976716
Pass：5, Batch：850, Cost：0.021980, Accuracy：0.975184
Pass：5, Batch：900, Cost：0.012631, Accuracy：0.976716
Pass：5, Batch：950, Cost：0.013144, Accuracy：0.976103
Pass：5, Batch：1000, Cost：0.014421, Accuracy：0.974877
Pass：5, Batch：1050, Cost：0.021123, Accuracy：0.974877
Pass：5, Batch：1100, Cost：0.019840, Accuracy：0.973652
Pass：5, Batch：1150, Cost：0.013162, Accuracy：0.976409
Pass：5, Batch：1200, Cost：0.016105, Accuracy：0.976409
Pass：5, Batch：1250, Cost：0.015523, Accuracy：0.975184
Pass：5, Batch：1300, Cost：0.019663, Accuracy：0.974265
test - Pass: 5 Cost: 0.015125105566247648 Acc: 0.9954541127189642
new BestPrecent: 0.9961800266565118 thie Precent: 0.9954541127189642

Pass：6, Batch：50, Cost：0.023014, Accuracy：0.993260
Pass：6, Batch：100, Cost：0.008408, Accuracy：0.978860
Pass：6, Batch：150, Cost：0.

test - Pass: 10 Cost: 0.012615335657526107 Acc: 0.9959182216298553
new BestPrecent: 0.9961800266565118 thie Precent: 0.9959182216298553

Pass：11, Batch：50, Cost：0.011193, Accuracy：0.996324
Pass：11, Batch：100, Cost：0.010670, Accuracy：0.976716
Pass：11, Batch：150, Cost：0.012213, Accuracy：0.976103
Pass：11, Batch：200, Cost：0.015350, Accuracy：0.976103
Pass：11, Batch：250, Cost：0.010715, Accuracy：0.976409
Pass：11, Batch：300, Cost：0.010645, Accuracy：0.977941
Pass：11, Batch：350, Cost：0.015909, Accuracy：0.976103
Pass：11, Batch：400, Cost：0.011423, Accuracy：0.977941
Pass：11, Batch：450, Cost：0.015543, Accuracy：0.975490
Pass：11, Batch：500, Cost：0.011441, Accuracy：0.976409
Pass：11, Batch：550, Cost：0.014235, Accuracy：0.976409
Pass：11, Batch：600, Cost：0.013267, Accuracy：0.975490
Pass：11, Batch：650, Cost：0.012854, Accuracy：0.975797
Pass：11, Batch：700, Cost：0.009532, Accuracy：0.977022
Pass：11, Batch：750, Cost：0.019862, Accuracy：0.973958
Pass：11, Batch：800, Cost：0.013608, Accuracy：0.975797
Pass：11, Batch：8

Pass：16, Batch：500, Cost：0.013584, Accuracy：0.977022
Pass：16, Batch：550, Cost：0.014516, Accuracy：0.975184
Pass：16, Batch：600, Cost：0.013820, Accuracy：0.975797
Pass：16, Batch：650, Cost：0.014284, Accuracy：0.974877
Pass：16, Batch：700, Cost：0.013030, Accuracy：0.976103
Pass：16, Batch：750, Cost：0.015714, Accuracy：0.976103
Pass：16, Batch：800, Cost：0.014877, Accuracy：0.975184
Pass：16, Batch：850, Cost：0.016759, Accuracy：0.974571
Pass：16, Batch：900, Cost：0.010743, Accuracy：0.977328
Pass：16, Batch：950, Cost：0.018255, Accuracy：0.973958
Pass：16, Batch：1000, Cost：0.009878, Accuracy：0.977328
Pass：16, Batch：1050, Cost：0.014540, Accuracy：0.975490
Pass：16, Batch：1100, Cost：0.016856, Accuracy：0.975184
Pass：16, Batch：1150, Cost：0.014834, Accuracy：0.975490
Pass：16, Batch：1200, Cost：0.013565, Accuracy：0.975184
Pass：16, Batch：1250, Cost：0.012982, Accuracy：0.975797
Pass：16, Batch：1300, Cost：0.011932, Accuracy：0.977635
test - Pass: 16 Cost: 0.011970430601265018 Acc: 0.9962276275704494
new BestPrecent: 0.996180

Pass：21, Batch：1050, Cost：0.010459, Accuracy：0.976716
Pass：21, Batch：1100, Cost：0.008947, Accuracy：0.977941
Pass：21, Batch：1150, Cost：0.013077, Accuracy：0.977328
Pass：21, Batch：1200, Cost：0.014446, Accuracy：0.975184
Pass：21, Batch：1250, Cost：0.012416, Accuracy：0.976716
Pass：21, Batch：1300, Cost：0.014761, Accuracy：0.975490
test - Pass: 21 Cost: 0.012930293407681892 Acc: 0.9960491241431836
new BestPrecent: 0.9962276275704494 thie Precent: 0.9960491241431836

Pass：22, Batch：50, Cost：0.017865, Accuracy：0.994485
Pass：22, Batch：100, Cost：0.013044, Accuracy：0.975490
Pass：22, Batch：150, Cost：0.013159, Accuracy：0.975490
Pass：22, Batch：200, Cost：0.019447, Accuracy：0.974877
Pass：22, Batch：250, Cost：0.013564, Accuracy：0.975797
Pass：22, Batch：300, Cost：0.019528, Accuracy：0.973958
Pass：22, Batch：350, Cost：0.019298, Accuracy：0.974265
Pass：22, Batch：400, Cost：0.013965, Accuracy：0.976103
Pass：22, Batch：450, Cost：0.018781, Accuracy：0.974265
Pass：22, Batch：500, Cost：0.007800, Accuracy：0.977328
Pass：22, B

In [ ]:
save_model(save_model_name)

In [ ]:
load_model(save_model_name)

In [ ]:
save_inference(save_model_name)

In [ ]:
test_image = test_list[19][20]
plt.figure()
plt.imshow(test_image)
test_image = np.array(test_image).astype('float32').reshape(1,1,64,64)/255
result = exe.run(program = test_program,feed = {'image':test_image,'label':np.array([[1]]).astype('int64')},fetch_list = [net])
print(np.argmax(result))